# # This version is modified to measure I,Q and sync from FG, then to demodulate. I modified the 3001 box to actually read ad0,ad1,ad2,ad3 on 2/8/2017

In [1]:
import numpy as np
import time
import sys
import os
sys.path.append('C:\\git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('C:\\git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('C:\git_repos\lab_utilities\IO_3001_USB_acquisition')
sys.path.append('C:\\Anaconda3\\envs\\py27\\Scripts')
from daq import daqDevice
import daqh
from prm_util import nps
import h5py
%pylab

import prm_util as cu

from Tkinter import Tk
from tkFileDialog import askopenfilename

acqmode=daqh.DaamInfinitePost

import gc
    

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
rcParams['agg.path.chunksize'] = 100000

In [3]:
dev=daqDevice('DaqBoard3001USB')

In [4]:
def set_dac(chan=0,dataVal=3.0):
    """
    function to simply set one da channel
    """
    devicetype=0
    dev=daqDevice('DaqBoard3001USB')
    dev.DacWt(devicetype,0,dataVal)
    dev.Close()    
    return 

In [5]:
def get_data(nchan=4,freq=10,nseconds=5,comment='None',alerts=[58,59,60,118,119,120,178,179,180,238,239,240,298,299,300]):
    """
    function to simply aquire nchan a/d channels at rate freq
    for nseconds seconds
    """
    DafSettle1us=0x1800000
    #outdata=np.zeros([nchan,nscans],dtype=float)
    dev=daqDevice('DaqBoard3001USB')
    gains=[]
    flags=[]
    chans=[]
    if nchan > 8:
        uchan=nchan-8
        for i in range(8):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential|DafSettle1us)
            chans.append(i)
        for i in range(uchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential|DafSettle1us)
            chans.append(256+i)   #HERE is the famous fix where DaqX refs upper level dif channels!
    elif nchan<9:      
        for i in range(nchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential|DafSettle1us)
            chans.append(i)
    acqmode = daqh.DaamNShot
    dev.AdcSetAcq(acqmode, postTrigCount = np.uint(nseconds*freq))
    dev.AdcSetScan(chans,gains,flags)
    dev.AdcSetFreq(freq)
    #use the driver buffer here user buffer was very limited (the way I tried anyway) 
    transMask = daqh.DatmUpdateBlock|daqh.DatmCycleOn|daqh.DatmDriverBuf

    buf=dev.AdcTransferSetBuffer(transMask, np.uint(nseconds*freq*nchan))
    #bufMask is for transferring the buffer
    bufMask = daqh.DabtmOldest | daqh.DabtmRetAvail

    timestart = (time.time())
    timenotify = timestart + 5

    dev.SetTriggerEvent(daqh.DatsImmediate,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStartEvent)
    dev.SetTriggerEvent(daqh.DatsScanCount,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStopEvent)
    dev.AdcTransferStart()
    dev.AdcArm()
    
    while True:
        alertscopy=alerts[:]
        timenotify = checkAlerts(timenotify, timestart, alerts,alertscopy)        
        
        stat = dev.AdcTransferGetStat()
        active = stat['active']
        if not (active & daqh.DaafAcqActive):
            break
    dev.AdcDisarm()
    outdata,ret=dev.AdcTransferBufData(np.uint(nseconds*freq), nchan,bufMask)
    
    outdata=np.array(outdata,dtype=np.uint16)
    #outdata=(outdata-2**15)*20./2**16
    outdata=np.transpose(np.reshape(outdata,[uint(nseconds*freq),nchan]))
    #print "Finished collecting data\n-"
    dev.Close()    
    return outdata

In [6]:
def checkAlerts(timeupdate,timestart, alerts, alertscopy, updateincrement=5):
    timecheck = (time.time())
    if timecheck > timeupdate:
        print "Time: " + str(int(timecheck - timestart))
        timeupdate += updateincrement
    for alert in alerts:
        if (timecheck - timestart)%60 > alert:
            print "---- " + str(int(timecheck - timestart)) + " SECONDS ----"
            alerts.remove(alert)
    if 2<(timecheck -timestart%60)<3:
        alerts = alertscopy[:]
        
        
    return timeupdate

In [7]:
def get_date_filename(labelstring=''):
    #version for testing adds an optional string to identify tests
    now=time.localtime()[0:6]
    dirfmt = "C:\\Interferometer_tests\\data\\%4d_%02d_%02d"
    dirname = dirfmt % now[0:3]
    filefmt = "%02d_%02d_%02d.h5"
    filename= labelstring+filefmt % now[3:6]
    ffilename=os.path.join(dirname,filename)
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    return(ffilename)

In [8]:
def zplot(d_dict_list,minfreq=1,chan=0):
    """
    calculate asd's and overplot all from list of data dictionaries, assumed to have meaningful
    keynames for legend
    """
    figure()
    for d_dict in d_dict_list:
        if len(d_dict['data'])>2:
            z=cu.nps(d_dict['data'],d_dict['samprate'],minfreq=minfreq)
        else:
            z=cu.nps(d_dict['data'][chan],d_dict['samprate'],minfreq=minfreq)
        plot(z[0],np.sqrt(z[1]),label=d_dict['label'])
    xlabel('Frequency, [Hz]')
    ylabel('ASD, [$nV \sqrt{sec}$]')
    xscale('log')
    yscale('log')
    legend()
    title('Comparison spectra')
    legend()

In [70]:
#use this function- box is set up with terminations on alternate channels to increase isolation
def get_test3(samprate=250000,nseconds=5,labelstring='test'):
    s=[0,1,2]  #active channels
    d=get_data(nchan=3,freq=samprate,nseconds=nseconds)
    outdata=squeeze(d[s,:])
    fname=get_date_filename(labelstring)
    with h5py.File(fname, mode='w') as hdf_file:
        hdf_file.create_dataset('data', data=outdata)
        hdf_file.create_dataset('samprate', data=[samprate])
        hdf_file.create_dataset('label', data=[labelstring])
        hdf_file.close()
    return {'data':outdata,'samprate':samprate,'label':labelstring}

In [10]:
def i2v(inarray):
    """convenient conversion to float from int16
    """
    return (inarray.astype(float)-2**15)*20./2**16
    

In [85]:
def demod(indata):
    #function to extract I,Q from data, sync, assume sync is in A/D ch 2 
    dd=i2v(indata['data'][2,4:]) - i2v(indata['data'][2,:-4])
    sv=i2v(indata['data'][0,4:])
    sh=i2v(indata['data'][1,4:])
    x=arange(len(dd))
    d_edges=x[where(dd > 2.)]
    u_edges=x[where(dd < -2.)]
    #trim these lists to avoid partials at the end, and force first edge to be rising 
    #last edge to be falling one
    u_edges=u_edges[1:-1]
    d_edges=d_edges[where(d_edges > u_edges[0])]
    d_edges=d_edges[where(d_edges < u_edges[-1])]
    i_v=[]
    q_v=[]
    i_h=[]
    q_h=[]
    print(len(d_edges),len(u_edges))
    ntot=0.0
    nchop=0.0
    for u,d,u2 in zip(u_edges[:-1],d_edges,u_edges[1:]):
        if(((d-u)>3) and ((d-u) <500) and ((u2-d>3)) and ((u2-d) <500)):
            nchop=nchop+1
            ntot=ntot+d-u
            i_v.append(np.mean(sv[u:d]))
            q_v.append(np.mean(sv[d:u2]))
            i_h.append(np.mean(sh[u:d]))
            q_h.append(np.mean(sh[d:u2]))
    nperchop=2*ntot/nchop
    print(ntot,nperchop)
    demod_samprate=indata['samprate']/nperchop
    i_v=np.array(i_v)
    q_v=np.array(q_v)
    i_h=np.array(i_h)
    q_h=np.array(q_h)
    z=i_v-mean(i_v) + 1j*(q_v-mean(q_v))
    phi_v=np.unwrap(np.angle(z))
    z=i_h-mean(i_h) + 1j*(q_h-mean(q_h))
    phi_h=np.unwrap(np.angle(z))    
    return {'i_v':i_v,'q_v':q_v,'i_h':i_h,'q_h':q_h,'phi_v':phi_v,'phi_h':phi_h,'samprate':demod_samprate}


In [68]:
def demod_allplot(d,closef=True,plotphase=True,plotxy=True,plottoi=False,plotfft=True,norm=True):
    '''
    This version of allplot demodulates first, assuming LiNbO3 modulator quadrature, two inputs, and synch a/d 2
    set closef to False to overplot on previous figures This version starts with integer array inputs and casts on the fly
    '''
    gc.enable()
    minfreq=.2
    dd=demod(d)
    sr=np.float(dd['samprate'])
    x=arange(len(dd['i_v']))/sr
    if closef:
        close(1)
        close(2)
        close(3)
        close(4)
        gc.collect()
    if plotphase:
        figure(1)
        plot(x,dd['phi_v']-dd['phi_v'][0],label=d['label']+' Vphase')
        plot(x,dd['phi_h']-dd['phi_h'][0],label=d['label']+' Hphase')
        xlabel('Time [seconds]')
        ylabel('Phase [Radians]')
        title(d['label'] + ' Phase')
        legend()
    if plotxy:
        figure(2)
        plot(dd['i_v'],dd['q_v'],'.',label='Vert')
        plot(dd['i_h'],dd['q_h'],'.',label='Hor')
        xlabel('I')
        ylabel('Q')
        title(d['label']+ ' Complex plane')
        legend()
    if plottoi:
        figure(3)
        plot(x,dd['i_v'],label='I_v '+d['label'])
        plot(x,dd['q_v'],label='Q_v '+d['label'])
        plot(x,dd['i_v'],label='I_v '+d['label'])
        plot(x,dd['q_v'],label='Q_v '+d['label'])
        legend()
        xlabel('Time [Seconds]')
        ylabel('Signal, [au]')
        title(d['label'])
        legend()
    if plotfft:
        figure(4)
        zphi_v=cu.nps(dd['phi_v'],dd['samprate'])
        plot(zphi_v[0],np.sqrt(zphi_v[1]),label='V_phase  '+d['label'])
        zphi_h=cu.nps(dd['phi_h'],dd['samprate'])
        plot(zphi_h[0],np.sqrt(zphi_h[1]),label='H_phase  '+d['label'])        
        xlabel('Frequency, [Hz]')
        ylabel('ASD, [$Rad/ \sqrt{Hz}$]')
        xscale('log')
        yscale('log')
        title('Phase spectrum')
        legend()

In [12]:
def import_data(filename=None):
    '''
    function to read in previously stored data and put in format of freshly read data dictionary
    '''
    if filename==None:
        Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
        filename = askopenfilename(initialdir='C:\\Interferometer_tests\\data',parent=Tk())
    hf=h5py.File(filename)
    outdata=reshape(hf['data'][:],[2,-1])
    samprate=hf['samprate'][0]
    labelstring=hf['label'][0]
    hf.close()
    return {'data':outdata,'samprate':samprate,'label':labelstring}

In [66]:
t6=get_test3(nseconds=10,labelstring='LiNbO_55Omv_5khz_pz1v_10hz')

Time: 5
Time: 10


In [80]:
dt5=demod(t5)

(39970, 39964)
(426077.0, 244.87183908045978)


In [81]:
figure()
plot(t5['data'][0])

In [88]:
demod_allplot(t6)

(199871, 199874)
(837683.0, 46.46308724832215)


In [56]:
dt4['samprate']

1015.7099966524395

In [45]:
figure()
plot(iv)
plot(qv)
plot(ih)
plot(qh)
figure()
plot(phiv-phiv[0])
plot(phih-phih[0])

In [15]:
fiber_pol_25km_smf_7=get_test2(nseconds=100,samprate=250000,labelstring='fiber_pol_splitter_25km_smf_pz80v_1kHz')

Time: 5
Time: 10
Time: 15
Time: 20
Time: 25
Time: 30
Time: 35
Time: 40
Time: 45
Time: 50
Time: 55
Time: 60
Time: 65
Time: 70
Time: 75
Time: 80
Time: 85
Time: 90
Time: 95
Time: 100
Finished collecting data
-


In [31]:
allplot(DUT25km_smf28_pz20Hz_20V_100s)

In [28]:
shape(fiber_pol_25km_smf_6['data'])

(2L, 25000000L)

In [29]:
2*25*8

400

In [21]:
allplot(tst1,plotxy=False)

In [29]:
close('all')
gc.collect()

26799

In [27]:
jnk


array([[32755, 32774, 32772, ..., 32770, 32774, 32771],
       [32768, 32763, 32767, ..., 32765, 32763, 32766],
       [32745, 32758, 32762, ..., 32759, 32755, 32758],
       [32766, 32767, 32766, ..., 32766, 32766, 32763]], dtype=uint16)

In [29]:
phi=getphase(jnk)
phi[0]

-2.47076571034947

In [31]:
def getphase(d):
    """function to return array of phase in radians from standard get_data dataset, assumes real data in ch0,ch2"""
    v0=i2v(d[0,:])
    v1=i2v(d[2][:])
    i=v0-np.mean(v0)
    q=v1-np.mean(v1)
    i=i/np.std(i)
    q=q/np.std(q)
    z=i-mean(i) + 1j*(q-mean(q))
    return(np.mean(np.unwrap(np.angle(z))))

In [59]:
def getspitphase(vgain=0.1, nseconds=10,rate=1000):
    """
    function to acquire data at 250 KHz for 1/rate at a time, calculate phase, and send out to D/A at 0.1V/rad
    """
    phi=[]
    for s in range(10):
        d=get_data(freq=250000,nseconds=.001)
        #phi.append(getphase(d))
        #set_dac(dataVal=vgain*phi[-1])
    return(phi)
        
    
        
        

In [60]:
getspitphase(nseconds=2)

[]